In [20]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import chardet

ModuleNotFoundError: No module named 'chardet'

In [18]:
# データベース接続
conn = sqlite3.connect('jalan.db')
cur = conn.cursor()

# テーブル作成
cur.execute('''
  CREATE TABLE IF NOT EXISTS inns (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    rating REAL NOT NULL,
    comment_count INTEGER NOT NULL
  )
''')

In [19]:
# スクレイピング対象URL
url = "https://www.jalan.net/110000/LRG_110500/?stayYear=2025&stayMonth=1&stayDay=22&stayCount=1&roomCount=1&adultNum=2&ypFlg=1&kenCd=110000&screenId=UWW1380&roomCrack=200000&lrgCd=110500&distCd=01&rootCd=04"

# データ取得
response = requests.get(url)
# エンコーディングを自動検出
detected_encoding = chardet.detect(response.content)['encoding']

# BeautifulSoupで解析、エンコーディングを指定
soup = BeautifulSoup(response.content, "html.parser", from_encoding=detected_encoding)

# 旅館情報を含む要素を取得
inns = soup.find_all("div", class_="p-yadoCassette__summary")

for inn in inns:
    try:
        name = inn.find("h2", class_="p-searchResultItem__facilityName").text.strip()
        rating = float(inn.find("span", class_="p-searchResultItem__summaryaverage-num").text)
        comment_count_text = inn.find("span", class_="p-searchResultItem__summarykuchikomi__totalNumber").text
        # 数値だけを抽出
        comment_count_match = re.search(r'\d+', comment_count_text)
        if comment_count_match:
            comment_count = int(comment_count_match.group(0))
        else:
            comment_count = 0 # 数値が見つからない場合は0を設定        
        # データベースに挿入
        cur.execute("INSERT INTO inns (name, rating, comment_count) VALUES (?, ?, ?)", (name, rating, comment_count))

    except AttributeError:
        print("Error: Could not find element. Skipping...")
        # エラーが発生した旅館の情報をprintするなどして、原因を調査する
        print(inn)

NameError: name 'chardet' is not defined

In [9]:
# コミット
conn.commit()

# データベース接続を閉じる
conn.close()